In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import re
from functools import partial, reduce
import datetime

In [ ]:
hv.extension("bokeh")
%load_ext autoreload
%autoreload 2
import janelle

In [ ]:
service = janelle.get_calendar_service()

In [ ]:
events = janelle.get_calendar_events(service, janelle.CALENDAR_NAME)

In [ ]:
for num in (1, 2, 3, 4):
    events["Ti{}".format(num)] = events["summary"].str.contains(
        "ti(?:| |-|#){}".format(num), case=False
    )

In [ ]:
events["num_scopes"] = events[["Ti1", "Ti2", "Ti3", "Ti4"]].sum(axis=1)

In [ ]:
MICROSCOPES = ("Ti1", "Ti2", "Ti3", "Ti4")

In [ ]:
USERS = {}
for u in (
    "Jacob",
    "Somenath",
    "Emanuele",
    "Antoine",
    "Janelle",
    "Silvia",
    "Suyang",
    "Luis",
    "Charles",
    "Pablo",
    "Juan",
    "Scott",
    "Daniel",
    "Laurent",
    "Sadik",
    "Ghee",
    "Burak",
    "Yoon",
    "Meriem",
    "Dirk",
    "Stephan",
    "Brandon",
    "Jeehae",
):
    USERS[u] = u
USERS.update({"Nate": "Nate|Nathan", "César": "Cesar|César", "Tom": "Tom|Thomas"})

In [ ]:
def find_users(users, event):
    u = [
        user
        for user, pattern in users.items()
        if re.search(pattern, event["summary"], re.IGNORECASE)
    ]
    if not u:
        u = [
            user
            for user, pattern in users.items()
            if re.search(pattern, event["creator"], re.IGNORECASE)
        ]
    return u

In [ ]:
events["users"] = events.apply(partial(find_users, USERS), axis=1)
events["num_users"] = events["users"].map(len)

In [ ]:
if True:
    factor = 1
else:
    factor = 1 / events["num_users"]
user_weights = pd.DataFrame(
    {user: events["users"].map(lambda x: user in x) * factor for user in USERS.keys()}
)

In [ ]:
def to_dayfrac(df):
    return df.applymap(lambda x: x.total_seconds() / (3600 * 24))

In [ ]:
def tally_duration(events, user_weights, dates):
    return pd.DataFrame(
        {
            start_date: user_weights.multiply(
                ((start_date <= events["end"]) & (events["start"] < end_date))
                * (events["end"] - events["start"]),
                axis=0,
            ).sum(axis=0)
            for start_date, end_date in zip(dates[:-1], dates[1:])
        }
    ).T

In [ ]:
def tally_upcoming(events, user_weights, date=None):
    if date is None:
        date = datetime.datetime.now()
    return user_weights.multiply(
        (events["start"] >= date) * (events["end"] - events["start"]), axis=0
    ).sum(axis=0)

In [ ]:
def busy_days(events, dates):
    all_days = reduce(
        lambda x, y: x | y,
        events.apply(
            lambda x: set(
                pd.date_range(start=x["start"], end=x["end"], normalize=True).values
            ),
            axis=1,
        ).values,
    )
    return pd.Series(
        {
            start_date: len(
                set(
                    pd.date_range(
                        start=start_date,
                        end=end_date,
                        freq="D",
                        closed="left",
                        normalize=True,
                    ).values
                )
                & all_days
            )
            for start_date, end_date in zip(dates[:-1], dates[1:])
        }
    ).T

# Styling

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import to_hex

cmap = cm.get_cmap("YlOrBr")

In [ ]:
def _lookup_range(range_to_color, value):
    for (start, stop), position in range_to_color.items():
        if start <= value < stop:
            return position
    return None


def highlight_ranges(range_to_color, cmap, data):
    """
    highlight the maximum in a Series or DataFrame
    """
    # color = to_hex(cmap(0.5))
    attr = "background-color: {}"
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        # is_max = data == data.max()
        # return [attr if v else '' for v in is_max]
        styles = []
        for d in data:
            position = _lookup_range(range_to_color, d)
            if position:
                styles.append(attr.format(to_hex(cmap(position))))
            else:
                styles.append("")
        return styles
    # else:  # from .apply(axis=None)
    # is_max = data == data.max().max()
    # return pd.DataFrame(np.where(is_max, attr, ''),
    #                    index=data.index, columns=data.columns)

# Output

In [ ]:
dates = pd.date_range(start="1/1/2011", end=datetime.datetime.now(), freq="MS")
dates2 = pd.date_range(start="1/1/2017", end=datetime.datetime.now(), freq="MS")

In [ ]:
obj = {
    microscope: hv.Curve(
        busy_days(events[events[microscope]], dates)
        .reset_index()
        .rename(columns={0: "days", "index": "date"})
    ).options(tools=["hover"])
    for microscope in MICROSCOPES
}
hv.NdOverlay(obj).redim(Element="microscope").options(width=900)

In [ ]:
durs = tally_duration(events, user_weights, dates)
dur_days = to_dayfrac(durs)
obj = {
    name: hv.Curve(d.reset_index().values)
    .redim(x="date", y="days")
    .options(tools=["hover"])
    for name, d in dur_days.iteritems()
}
hv.NdOverlay(obj).redim(Element="name").options(width=900)

In [ ]:
recent_durs = to_dayfrac(
    tally_duration(events[events["all_day"]], user_weights, dates2)
)
total_durs = recent_durs.sum(axis=0).sort_values(ascending=False)
recent_durs = recent_durs.reindex(columns=total_durs.index).loc[:, total_durs > 0]
# recent_durs.style.set_precision(2).highlight_max(axis=1)#.background_gradient(cmap='viridis', low=0.7, high=0)
recent_durs.style.set_precision(2).apply(
    partial(
        highlight_ranges,
        {(3, 6): 0.1, (6, 10): 0.2, (10, 15): 0.3, (15, 20): 0.5, (20, 32): 0.6},
        cmap,
    ),
    axis=1,
)

In [ ]:
to_dayfrac(
    tally_upcoming(events[events["all_day"]], user_weights).to_frame()
).sort_values(0, ascending=False).T.style.set_precision(2).apply(
    partial(
        highlight_ranges,
        {(3, 6): 0.1, (6, 10): 0.2, (10, 15): 0.3, (15, 20): 0.5, (20, 32): 0.6},
        cmap,
    ),
    axis=1,
).hide_index()

In [ ]:
{microscope: hv.Curve(busy_days(events[events[microscope]], dates).reset_index().rename(columns={0: 'days', 'index': 'date'}))